In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import gc
import lightgbm as lgb
plt.rcParams['figure.figsize'] = [20, 8]
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
y = train['target']
train.drop(['ID_code', 'target'], axis=1, inplace=True)
test.drop(['ID_code'], axis=1, inplace=True)
gc.collect()

****Set different parameters for 5 models. The parameters are chosen almost randomly. Also, we have the same size of the training and test sample, so we will make a prediction for them at the same time. You can add different models, with different parameters and also add more layers before the final prediction.****

In [ ]:
param1 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}
param2 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 7,
                 'max_depth': 20,
                 'reg_alpha': 0.1,
                 'reg_lambda': 0.1,
                 'learning_rate' : 0.2,}
param3 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 20,
                 'max_depth': 15,
                 'reg_alpha': 0,
                 'reg_lambda': 0,
                 'learning_rate' : 0.2,}
param4 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 20,
                 'max_depth': 25,
                 'reg_alpha': 0.5,
                 'reg_lambda': 0.5,
                 'learning_rate' : 0.15,}
param5 = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'num_leaves': 30,
                 'max_depth': 100,
                 'reg_alpha': 0.977,
                 'reg_lambda': 0.411,
                 'learning_rate' : 0.083,
                 'min_data_in_leaf':1749,
                 'bagging_fraction ':0.645,
                 'bagging_fraq': 11,
                 'feature_fraction':0.458,
                 'verbose': 200,
                 'min_gain_to_split':0.542,
                 'subsample' : 0.376,
                 'max_bin':638}

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb

kf = KFold(n_splits=3, random_state = 42,shuffle = True)

oof_clf1 = np.zeros(len(train))
oof_clf2 = np.zeros(len(train))
oof_clf3 = np.zeros(len(train))
oof_clf4 = np.zeros(len(train))
oof_clf5 = np.zeros(len(train))

oof_clf1_t = np.zeros(len(train))
oof_clf2_t = np.zeros(len(train))
oof_clf3_t = np.zeros(len(train))
oof_clf4_t = np.zeros(len(train))
oof_clf5_t = np.zeros(len(train))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(train.values, y.values)):
        print("fold n°{}".format(fold_))
        
        train_data = lgb.Dataset(train.iloc[trn_idx], label = y.iloc[trn_idx])
        valid_data = lgb.Dataset(train.iloc[val_idx], label=y.iloc[val_idx])
        
        model1 = lgb.train(param1, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model2 = lgb.train(param2, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model3 = lgb.train(param3, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model4 = lgb.train(param4, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
        model5 = lgb.train(param5, train_data, 15000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)            
        
        oof_clf1[val_idx] = model1.predict(train.iloc[val_idx])
        oof_clf2[val_idx] = model2.predict(train.iloc[val_idx])
        oof_clf3[val_idx] = model3.predict(train.iloc[val_idx])
        oof_clf4[val_idx] = model4.predict(train.iloc[val_idx])
        oof_clf5[val_idx] = model5.predict(train.iloc[val_idx])
        
        oof_clf1_t[val_idx] = model1.predict(test.iloc[val_idx])
        oof_clf2_t[val_idx] = model2.predict(test.iloc[val_idx])
        oof_clf3_t[val_idx] = model3.predict(test.iloc[val_idx])
        oof_clf4_t[val_idx] = model4.predict(test.iloc[val_idx])
        oof_clf5_t[val_idx] = model5.predict(test.iloc[val_idx])
        
        print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1[val_idx])))
        print("AUC LGB2:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf2[val_idx])))
        print("AUC LGB3:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf3[val_idx])))
        print("AUC LGB4:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf4[val_idx])))
        print("AUC LGB5:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf5[val_idx]))) 

****Now we will make a data frame with the predictions of the previous models, on which we will make the main prediction****

In [ ]:
f_layer_train = pd.DataFrame( {'LGBM1': oof_clf1.ravel(),
     'LGBM2': oof_clf2.ravel(),
     'LGBM3': oof_clf3.ravel(),
      'LGBM4' :oof_clf4.ravel(),
        'LGBM5': oof_clf5.ravel(),
    })

f_layer_test = pd.DataFrame( {'LGBM1': oof_clf1_t.ravel(),
     'LGBM2': oof_clf2_t.ravel(),
     'LGBM3': oof_clf3_t.ravel(),
      'LGBM4' :oof_clf4_t.ravel(),
        'LGBM5': oof_clf5_t.ravel(),
    })

In [ ]:
param = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}

In [ ]:
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

oof_clf1_3 = np.zeros(len(train))
oof_clf1_t_3 = np.zeros(len(train))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(f_layer_train.values, y.values)):
        print("fold n°{}".format(fold_))
        train_data = lgb.Dataset(f_layer_train.iloc[trn_idx], label = y.iloc[trn_idx])
        valid_data = lgb.Dataset(f_layer_train.iloc[val_idx], label=y.iloc[val_idx])
        
        model = lgb.train(param, train_data, 10000, valid_sets=[valid_data], early_stopping_rounds=2000,verbose_eval=100)
                
        oof_clf1_3[val_idx] = model.predict(f_layer_train.iloc[val_idx])        
        
        oof_clf1_t_3[val_idx] = model.predict(f_layer_test.iloc[val_idx])
        
        print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1_3[val_idx])))

In [ ]:
sbm = pd.read_csv('../input/sample_submission.csv', usecols=['ID_code'])
sbm['target'] = oof_clf1_t_3
sbm.head()

In [ ]:
sbm.to_csv('submission.csv', index=False)